In [4]:
import numpy as np
import pandas as pd
import pickle
import os
import sys
import psycopg2
from dotenv import load_dotenv

In [12]:
#dotenv 로딩
load_dotenv(verbose=True)

True

In [6]:
#env 파일로부터 변수 추출
HOST = os.getenv('postgre_host')
PASSWORD = os.getenv('postgre_password')

DATABASE = 'postgredb'
USERNAME = 'kjcheong'
PORT = 5432

In [7]:
#연결
try:
    conn = psycopg2.connect(
        host=HOST,
        port=PORT,
        database=DATABASE,
        user=USERNAME,
        password = PASSWORD)
    cur = conn.cursor()
    print('connection success to DB')
except:
    print('connection failure to DB')
    sys.exit()

connection success to DB


In [8]:
#DB를 dataframe으로 저장
import pandas.io.sql as psql
month_avg = psql.read_sql("SELECT * FROM w_avg_month", conn)
month_avg

C:\Users\dankj\AppData\Local\Temp\ipykernel_68164\2089608740.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  month_avg = psql.read_sql("SELECT * FROM w_avg_month", conn)


,month,season,avgta,maxta,minta,sumrn,avgws,avgrhm,sumsshr,avgps
0,1,winter,-1.9,2.1,-5.5,16.8,2.3,56.2,169.6,1024.9
1,2,winter,0.7,5.1,-3.2,28.2,2.5,54.6,170.8,1023.2
2,3,spring,6.1,11.0,1.9,36.9,2.7,54.6,198.2,1019.4
3,4,spring,12.6,17.9,8.0,72.9,2.7,54.8,206.3,1014.8
4,5,spring,18.2,23.6,13.5,103.6,2.5,59.7,223.0,1010.9
5,6,summer,22.7,27.6,18.7,129.5,2.2,65.7,189.1,1007.3
6,7,summer,25.3,29.0,22.3,414.4,2.2,76.2,123.6,1006.4
7,8,summer,26.1,30.0,22.9,348.2,2.1,73.5,156.1,1008.2
8,9,fall,21.6,26.2,17.7,141.5,1.9,66.4,179.7,1013.5
9,10,fall,15.0,20.2,10.6,52.2,2.0,61.8,206.5,1019.2


In [9]:
season_avg = psql.read_sql("SELECT * FROM w_avg_season", conn)
season_avg

C:\Users\dankj\AppData\Local\Temp\ipykernel_68164\1135474443.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  season_avg = psql.read_sql("SELECT * FROM w_avg_season", conn)


,season,avgta,maxta,minta,sumrn,avgws,avgrhm,sumsshr,avgps
0,spring,12.3,17.5,7.8,71.1,2.6,56.4,209.2,1015.0
1,summer,24.7,28.9,21.3,297.4,2.2,71.8,156.3,1007.3
2,fall,14.7,19.4,10.6,81.6,2.0,62.9,181.2,1018.4
3,winter,-0.3,3.8,-4.0,22.5,2.4,56.2,167.8,1024.4


In [10]:
#연결 종료
conn.close()

In [13]:
#예측에 쓰일 변수만 남기기
month_var = month_avg.iloc[:,2:]
display(month_var)
season_var = season_avg.iloc[:,1:]
display(season_var)

,avgta,maxta,minta,sumrn,avgws,avgrhm,sumsshr,avgps
0,-1.9,2.1,-5.5,16.8,2.3,56.2,169.6,1024.9
1,0.7,5.1,-3.2,28.2,2.5,54.6,170.8,1023.2
2,6.1,11.0,1.9,36.9,2.7,54.6,198.2,1019.4
3,12.6,17.9,8.0,72.9,2.7,54.8,206.3,1014.8
4,18.2,23.6,13.5,103.6,2.5,59.7,223.0,1010.9
5,22.7,27.6,18.7,129.5,2.2,65.7,189.1,1007.3
6,25.3,29.0,22.3,414.4,2.2,76.2,123.6,1006.4
7,26.1,30.0,22.9,348.2,2.1,73.5,156.1,1008.2
8,21.6,26.2,17.7,141.5,1.9,66.4,179.7,1013.5
9,15.0,20.2,10.6,52.2,2.0,61.8,206.5,1019.2


,avgta,maxta,minta,sumrn,avgws,avgrhm,sumsshr,avgps
0,12.3,17.5,7.8,71.1,2.6,56.4,209.2,1015.0
1,24.7,28.9,21.3,297.4,2.2,71.8,156.3,1007.3
2,14.7,19.4,10.6,81.6,2.0,62.9,181.2,1018.4
3,-0.3,3.8,-4.0,22.5,2.4,56.2,167.8,1024.4


In [3]:
#모델 복호화
model = pickle.load(open("model.pkl", "rb"))

In [15]:
#예측
predict_month = model.predict(month_var).round(0)
predict_season = model.predict(season_var).round(0)
print(predict_month)
print(predict_season)

[ 94.  34.  16.  14. 142. 222. 162. 171. 124.  80.  68. 149.]
[ 72. 188.  54.  52.]


In [18]:
#기존 dataframe에 column 추가
month_pred = month_avg.copy()
season_pred = season_avg.copy()
month_pred['predict_patient'] = predict_month.astype(int)
season_pred['predict_patient'] = predict_season.astype(int)
display(month_pred)
display(season_pred)

,month,season,avgta,maxta,minta,sumrn,avgws,avgrhm,sumsshr,avgps,predict_patient
0,1,winter,-1.9,2.1,-5.5,16.8,2.3,56.2,169.6,1024.9,94
1,2,winter,0.7,5.1,-3.2,28.2,2.5,54.6,170.8,1023.2,34
2,3,spring,6.1,11.0,1.9,36.9,2.7,54.6,198.2,1019.4,16
3,4,spring,12.6,17.9,8.0,72.9,2.7,54.8,206.3,1014.8,14
4,5,spring,18.2,23.6,13.5,103.6,2.5,59.7,223.0,1010.9,142
5,6,summer,22.7,27.6,18.7,129.5,2.2,65.7,189.1,1007.3,222
6,7,summer,25.3,29.0,22.3,414.4,2.2,76.2,123.6,1006.4,162
7,8,summer,26.1,30.0,22.9,348.2,2.1,73.5,156.1,1008.2,171
8,9,fall,21.6,26.2,17.7,141.5,1.9,66.4,179.7,1013.5,124
9,10,fall,15.0,20.2,10.6,52.2,2.0,61.8,206.5,1019.2,80


,season,avgta,maxta,minta,sumrn,avgws,avgrhm,sumsshr,avgps,predict_patient
0,spring,12.3,17.5,7.8,71.1,2.6,56.4,209.2,1015.0,72
1,summer,24.7,28.9,21.3,297.4,2.2,71.8,156.3,1007.3,188
2,fall,14.7,19.4,10.6,81.6,2.0,62.9,181.2,1018.4,54
3,winter,-0.3,3.8,-4.0,22.5,2.4,56.2,167.8,1024.4,52


In [19]:
#csv file로 저장
month_pred.to_csv('csv/pred-month.csv', index=False)
season_pred.to_csv('csv/pred-season.csv', index=False)